In [1]:
# import esm
import torch
from argparse import Namespace
from esm.constants import proteinseq_toks
import math
import torch.nn as nn
import torch.nn.functional as F
from esm.modules import TransformerLayer, PositionalEmbedding  # noqa
from esm.model import ProteinBertModel

# model, alphabet = torch.hub.load("facebookresearch/esm", "esm1_t34_670M_UR50S")
import esm

In [2]:
from ych_util import prepare_mlm_mask
import pandas as pd
import time

In [3]:
dat = pd.read_csv("../../data/kif/dyn_kin.csv")
dat = dat.sample(frac = 1)
dat.head()

,Entry,seq
1560,D3ZKU7,MLSRLLKEHQAKQNERKELQEKRRREAITAATCLTEALVDHLNVGV...
1260,Q9US60,MTSIKVVCRIRPTNQLEQDLGGNNVIYPLNDSTVHIETSDYSGNFV...
1422,P70248,MGSKERFHWQSHNVKQSGVDDMVLLSQITEDAIVRNLHKRFMDDYI...
1740,Q9LX99,MADQRSKTNRWNWEVSGFEPRKSSSNASFAESTGHRTTGPLLRRNS...
667,Q9FKP4,MAEQKSTNMWNWEVTGFESKKSPSSEEGVHRTPSSMLRRYSIPKNS...


In [4]:
alphabet = esm.Alphabet.from_dict(proteinseq_toks)
# model_name = "esm1_t34_670M_UR50S"
model_name = "esm1_t12_85M_UR50S"
url = f"https://dl.fbaipublicfiles.com/fair-esm/models/{model_name}.pt"
if torch.cuda.is_available():
    print("cuda")
    model_data = torch.hub.load_state_dict_from_url(url, progress=False)
else:
    model_data = torch.hub.load_state_dict_from_url(url, progress=False, map_location=torch.device('cpu'))

pra = lambda s: ''.join(s.split('decoder_')[1:] if 'decoder' in s else s)
prs = lambda s: ''.join(s.split('decoder.')[1:] if 'decoder' in s else s)
model_args = {pra(arg[0]): arg[1] for arg in vars(model_data["args"]).items()}
model_state = {prs(arg[0]): arg[1] for arg in model_data["model"].items()}

cuda


In [5]:
model = esm.ProteinBertModel(
        Namespace(**model_args), len(alphabet), padding_idx=alphabet.padding_idx
    )

model.load_state_dict(model_state)

<All keys matched successfully>

In [6]:
model.cuda()
model.train()

ProteinBertModel(
  (embed_tokens): Embedding(35, 768, padding_idx=1)
  (embed_positions): PositionalEmbedding()
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): BertLayerNorm()
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (final_layer_norm): BertLayerNorm()
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=Tr

In [7]:
batch_converter = alphabet.get_batch_converter()
criterion = nn.CrossEntropyLoss()
lr = 0.0001 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
start_time = time.time()
print_every = 100
for j in range(300):
    dat = dat.sample(frac = 1)
    for i in range(dat.shape[0]):
        if len(dat.iloc[i,1])>1024:
            seq = dat.iloc[i,1][:1023]
        else:
            seq = dat.iloc[i,1]
        lab = dat.iloc[i,0]
        data = [(lab, seq)]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        true_aa,target_ind,masked_batch_tokens = prepare_mlm_mask(alphabet,batch_tokens)
        optimizer.zero_grad()
        results = model(masked_batch_tokens.to('cuda'), repr_layers=[34])   
        pred = results["logits"].squeeze(0)[target_ind,:]   
        target = true_aa.squeeze(0)
        loss = criterion(pred.cpu(),target)
        loss.backward()
        optimizer.step()

        if i % print_every == 1:
            print(batch_labels)
            print(batch_strs)
            print(batch_tokens.size())
            print(masked_batch_tokens.size())
            print(results["logits"].size())
            print(pred.size())
            print(target.size())
            print(f"At Epoch: %.1f"% i)
            print(f"Loss %.4f"% loss)
            elapsed = time.time() - start_time
            print(f"time elapsed %.4f"% elapsed)
            torch.save(model.state_dict(), "../../data/esm_t12_85M_UR50S_dyn_kin_201204.pt")
#         break
#     break

['Q3V3Y6']
['MGTRKKVQAFVRVRPTDDFAHEMIKYGEDNKSIDIHLKKDTRRGVVNNQQTDWSFKLDGVLHNASQDLVYETVAKDAVSQALDGYNGTIMCYGQTGAGKTYTMTGATENYKHRGILPRALQQVFRMIEERPTHAITVRVSYLEIYNENLFDLLSTLPYVGPSVTPMTIVENPQGIFIKGLSVHLTSQEEDAFSLLFEGETNRIIASHTMNKNSSRSHCIFTIYMEAHSRTLSDEKYITSKINLVDLAGSERLSKTGSEGRVLKEATYINKSLSFLEQAIIALGDQNRDHVPFRQSKLTHALKDSLGGNCNMVLVTNIYGEAAQLDETLSSLRFASRMKLVTTEPAINEKYDAERMVKNLEKELALLKQELAIHDSLSNRTLVNYDPMDEIQIAEINSQVRRYLEGTLDEIDIINLRQIQEVFNQFRVVLSQQEQEVESALRRKYTLIDKNDFAAISAVQKVGLMDIEGNLVGEPDGQSFGLGVAPFSVKPGKKPKTKKTPKDQFSSSARKEGASSPVSGKDFDVASISKTQLIPSSKDGDLKDMLARERETSSIEPLISDSPKEELRAPRPSTPPSRTVAFEEFKNERGSEINRIFKENKSILNERKKRASETTQRINAIKQEIDETKDALNFQKSLREKQGEYENKGLMIIDEEEFLLILKLKDLKKQYRNEYQELRDLRAEIQYCQRLVDQCRHRLLMEFDIWYNESFMIPEDVQVALKLGSSIRPGMVPISRIVCLGEDDQDRFSHLQQTVLPEGLDSITFYNAKVKTDQKVTLDIHVCEGGGPEDFVIRILRTTNSIR']
torch.Size([1, 803])
torch.Size([1, 803])
torch.Size([1, 803, 35])
torch.Size([120, 35])
torch.Size([120])
At Epoch: 1.0
Loss 2.1879
time elapsed 1.1491
['P38401']
['MGCTLSAEDKAAVERS

['Q91636']
['MERLVATRLVTGLAVKIMRSNGVIHNANITSVNMDRSSVNVEWKEGEANKGKEISFADVISVNPELLDAVLAPTNVKENMPPQRNVSSQNHKRKTISKIPAPKEVAAKNSLLSESGAQSVLRERSTRMTAIHETLPYENEMEAESTPLPIQQNSVQARSRSTKVSIAEEPRLQTRISEIVEESLPSGRNNQGRRKSNIVKEMEKMKNKREEQRAQNYERRMKRAQDYDTSVPNWEFGKMIKEFRATMDCHRISMADPAEEHRICVCVRKRPLNKQELSKKEIDIISVPSKNIVLVHEPKLKVDLTKYLENQAFRFDFSFDETATNEVVYRFTARPLVQSIFEGGKATCFAYGQTGSGKTHTMGGDFSGKSQNVSKGVYAFASRDVFLLLDQPRYKHLDLDVFVTFFEIYNGKVFDLLNKKTKLRVLEDAKQEVQVVGLLEKQVISADDVFKMIEIGSACRTSGQTFANTSSSRSHACLQIILRRGSKLHGKFSLVDLAGNERGVDTASADRITRMEGAEINRSLLALKECIRALGQNKSHTPFRESKLTQILRDSFIGENSRTCMIAMLSPGFNSCEYTLNTLRYADRVKELSPQNAETNDDNLQMEDSGGSHASIEGLQLQDDFLLKDEELSTHNSFQDALNRVGELEDKAVDELRELVQKEPEWTNLLQMTEQPDYDLENFVMQAEYLIQERSKVLIALGDSINSLRLALQVEEQASKQISKKKRSNK']
torch.Size([1, 731])
torch.Size([1, 731])
torch.Size([1, 731, 35])
torch.Size([109, 35])
torch.Size([109])
At Epoch: 1001.0
Loss 2.3401
time elapsed 414.1124
['Q61771']
['MSKLKSSESVRVVVRCRPMNGKEKAASYDKVVDVDVKLGQVSVKNPKGTSHEMPKTFTFDAVYDWNAKQFELYDETFRPLVDS

['Q22799']
['MVDRKAVIKNADMSDDMQQDAIDCATQALEKYNIEKDIAAYIKKEFDKKYNPTWHCIVGRNFGSYVTHETKHFIYFYLGQVAILLFKSG']
torch.Size([1, 90])
torch.Size([1, 90])
torch.Size([1, 90, 35])
torch.Size([13, 35])
torch.Size([13])
At Epoch: 2001.0
Loss 2.5258
time elapsed 828.2094
['Q60LV4']
['MSGNPDAFENDPGFPFLGMSREAKAASAARPFDSKKNCWIPDPEDGFVAAEIQSTTGDQVTVVTVKGNQITVKKDQCQEMNPPKFDKTEDMANLTFLNEASVLGNLKDRYKDLMIYTYSGLFCVVINPYKRLPIYSESVIKHFMGKRRNEMPPHLFAVSDEAYRNMVQDKENQSMLITGESGAGKTENTKKVISYFAIVGATQAAAGGKKEEGKKGGTLEEQIVQTNPVLEAFGNAKTVRNNNSSRFGKFIRTHFSGSGKLAGGDIEHYLLEKSRVVRQAPGERCYHIFYQIMSGNDASLRGKLKLNNDITYYHFCSQAELTIEGMDDKEEMRLTQEAFDIMGFEDQETMDLYRSTAGIMHMGEMKFKQRPREEQAEPDGEEDALNAAAMLGINAEEFLKALTKPRVRVGTEWVNKGQNLEQVSWAVSGLAKAIYARMFKWIINRCNKTLDAKEIERKHFIGVLDIAGFEIFDLNSFEQLWINFVNERLQQFFNHHMFVLEQEEYKREGIAWTFIDFGLDLQACIELIEKPLGIISILDEECIVPKATDMTYAQKLLDQHLGKHPNFQKPKPPKGKQGDAHFAIVHYAGTVRYNANNFLEKNKDPLNDTAVALLKHSVDNNLMLDIWQDYQTQEEAAEAAKAGQSGGGKRGKSSSFATVSMIYRESLNNLMNMLYQTHPHFIRCIIPNEKKASGVIDSALVLNQLTCNGVLEGIRICRKGFPNRMLYPD

['Q715S9']
['MASKKREVQLQSVVNSQNLWDEMLLNKGLTVIDVYQAWCGPCKAVQALFRKLKNELNEDELLHFVVAEADSIVTLQPFRDKCEPVFLFSLNGKIIAKIQGANAPLINRKVIALIDEEKKIAAGEMARPQYVEIPLVDSLDEEYGEVHYESNVEVYNMAVINPDAVLMRKNLEIKEKITKEGFIIEIQENMLLPEEVAREFYNHMIDEPDFEEFVYSMTNRLSCVLIISQGEDTEVIEEEALPQSDDEEEPDPLEEPHVRFAPMLVKKKRDSLQEYMDRQHMSDYCHVEDDAVKVSKFIDILFPDFKTMKSTNVQRTLGLLYPEVCEEEKDNVLDIIQNEGFTILMQRQVVLSEEEARAVCHVHEDEDYFDNLIGYMCSNNSYILVLMREHSVERWKELIGPKTVEEAYASHPDSLCVRFASGNFPVNQFYGSSSKAAAETEIEHFFPPQSTLALIKPHVSHKERMEILKAIRDARFELTQMKEMHLTPEHASKVYFKITGKDFYKNVLDVLSSGMSVVMILTKWNAVGEWRRMMGPVDPEEAKLLSPNSLRARYGIDVLRNAVHGASNMSEAATAISNVFTESNFEN']
torch.Size([1, 588])
torch.Size([1, 588])
torch.Size([1, 588, 35])
torch.Size([88, 35])
torch.Size([88])
At Epoch: 701.0
Loss 2.7742
time elapsed 1240.5097
['A8E5U3']
['MGAEQSSEGDLRQNDLSSSVSPSPAKQKAKMDDIVVVAPGTQSLRNVSKDPEVIKLQEIPTFQPLLKGLLSGQTSPTNVRLERLDSPQVLQLCVRYQEHLHQCAEAVSFDQNALVKRIKEMDSSVDGLYSLMQERQKRFAKYAEQIQKVNEMSAILRRIQMGIDQTVPLMEKLNIMLPEGERLEPFCMSPDRESKP']
torch.Size([1, 197])
torch.Si

['O60296']
['MSQSQNAIFTSPTGEENLMNSNHRDSESITDVCSNEDLPEVELVSLLEEQLPQYRLKVDTLFLYENQDWTQSPHQRQHASDALSPVLAEETFRYMILGTDRVEQMTKTYNDIDMVTHLLAERDRDLELAARIGQALLKRNHVLSEQNESLEEQLGQAFDQVNQLQHELCKKDELLRIVSIASEESETDSSCSTPLRFNESFSLSQGLLQLEMLQEKLKELEEENMALRSKACHIKTETVTYEEKEQQLVSDCVKELRETNAQMSRMTEELSGKSDELIRYQEELSSLLSQIVDLQHKLKEHVIEKEELKLHLQASKDAQRQLTMELHELQDRNMECLGMLHESQEEIKELRSRSGPTAHLYFSQSYGAFTGESLAAEIEGTMRKKLSLDEESSLFKQKAQQKRVFDTVRIANDTRGRSISFPALLPIPGSNRSSVIMTAKPFESGLQQTEDKSLLNQGSSSEEVAGSSQKMGQPGPSGDSDLATALHRLSLRRQNYLSEKQFFAEEWQRKIQVLADQKEGVSGCVTPTESLASLCTTQSEITDLSSASCLRGFMPEKLQIVKPLEGSQTLYHWQQLAQPNLGTILDPRPGVITKGFTQLPGDAIYHISDLEEDEEEGITFQVQQPLEVEEKLSTSKPVTGIFLPPITSAGGPVTVATANPGKCLSCTNSTFTFTTCRILHPSDITQVTPSSGFPSLSCGSSGSSSSNTAVNSPALSYRLSIGESITNRRDSTTTFSSTMSLAKLLQERGISAKVYHSPISENPLQPLPKSLAIPSTPPNSPSHSPCPSPLPFEPRVHLSENFLASRPAETFLQEMYGLRPSRNPPDVGQLKMNLVDRLKRLGIARVVKNPGAQENGRCQEAEIGPQKPDSAVYLNSGSSLLGGLRRNQSLPVIMGSFAAPVCTSSPKMGVLKED']
torch.Size([1, 915])
torch.Size([1, 915])
torch.Size([1, 915, 35])
tor

...

['P0C279']
['MAFSLEEESGRIKDCWDNQEVPALSTCSNANIFRRINAILDDSLDFSKVCTTPINRGIHDQLPDFQDSEEAITSRMLFPTSAQESPRGLPDANGLCLGLQSLSLTGWDRPWSTQDSDSSAQSNTQSVLSMLQNPLGNVLGKTPLSFLSLDPLGSDLDKFPAPSVRGSRLDTRPILDSRSSSPSDSDTSGFSSGSDHLSDLISSLRISPPLPFLSMTGNGPRDPLKMGVGSRMDQEQAALAAVAPSPTSAPKRWPGTSVWPSWDLLGAPKDPFSIEREARLHRQAAAVNEATCTWSGQLPPRNYKNPIYSCKVFLGGVPWDITEAGLVNTFRVFGSLSVEWPGKDGKHPRCPPKGNMPKGYVYLVFELEKSVRALLQACSHDPLSPDGLSEYYFKMSSRRMRCKEVQVIPWVLADSNFVWSPSQRLDPSRTVFVGALHGMLNAEALAAILNDLFGGVVYAGIDTDKHKYPIGSGRVTFNNQRSYLKAVTAAFVEIKTTKFTKKVQIDPYLEDSLCLICSSQPGPFFCRDQVCFKYFCRSCWHWRHSMEGLRHHSPLMRNQKN']
torch.Size([1, 562])
torch.Size([1, 562])
torch.Size([1, 562, 35])
torch.Size([84, 35])
torch.Size([84])
At Epoch: 2101.0
Loss 0.0655
time elapsed 70370.9026
['Q9US03']
['MEEEGHKSLTSHLPQSSSSLSQSREIAKEFTSNIPPPTIKTNSSSSNILKPRLSLQNEVNQLKPAKFPSKMLPPGSLASVKSSSLAKKARPFTASSNPRMPKSAHPISSRSVSASSHFGRPASAVSSSLNSSDDVRSMSDESMESYNDEKSVNASALRTTEDRLRSMEMAYAQLSAKVIPSPSKRPANYKFYEQRIAMLEESLEVERSRTSELQEQFSVALREKAEAEANKIVSQKGMESLEIMLNSMKSEN

['Q5PQM2']
['MSGLVLGQRDEPAGHRLSQEEILGSTRLVSQGLESLHSEHQAVLQSLSHTIECLQQGGHEEGLVHEKARQLRRSMENIELGLSEAQVMLALASHLSTVESEKQKLRAQVRRLCQENQWLRDELAGTQQRLQRSEQAVAQLEEEKKHLEFLRQLRQYDEDGHSMEEKEGDASKDSLDDLFPNEEEEDSSNDLSRGQGAAAAQQGGYEIPARLRTLHNLVIQYAAQGRYEVAVPLCKQALEDLERTSGRGHPDVATMLNILALVYRDQNKYKEAAHLLNDALSIRESTLGRDHPAVAATLNNLAVLYGKRGKYKEAEPLCQRALEIREKVLGTDHPDVAKQLNNLALLCQNQGKYEAVERYYQRALAIYERQLGPDNPNVARTKNNLASCYLKQGKYSEAETLYKEILTRAHVQEFGSVDDDHKPIWMHAEEREEMSRSRSRESGTPYAEYGGWYKACRVSSPTVNTTLRNLGALYRRQGKLEAAETLEECALRSRKQGTDPISQTKVAELLGEGDGRKTMQEGPGDSVKFEGGEDASVAVEWSGDGSGTLQRSGSLGKIRDVLRRSSELLVRKLQGTEPRPSSSNMKRAASLNYLNQPNAAPLQTSRGLSASTVDLSSSS']
torch.Size([1, 620])
torch.Size([1, 620])
torch.Size([1, 620, 35])
torch.Size([92, 35])
torch.Size([92])
At Epoch: 901.0
Loss 0.2148
time elapsed 70824.5101
['A3BFT0']
['MSTRATRPGMLHQKENAADAQAGKRQRTAAGSAARAPLSANAAPPAPDPAIEFAGRDDVDALLNEKMKGKNKMDYKGKSEQMMEYIKKLRACIKWLLEREDTNLAEIGKLNGLLEAAEKHHSEIVAQLKSAIEESKAINEELQRQYASLEENLKRVEAEKLDALRSYGDEKEARIAVEASRNEHLEDLRRIKLEE

['P43034']
['MVLSQRQRDELNRAIADYLRSNGYEEAYSVFKKEAELDVNEELDKKYAGLLEKKWTSVIRLQKKVMELESKLNEAKEEFTSGGPLGQKRDPKEWIPRPPEKYALSGHRSPVTRVIFHPVFSVMVSASEDATIKVWDYETGDFERTLKGHTDSVQDISFDHSGKLLASCSADMTIKLWDFQGFECIRTMHGHDHNVSSVAIMPNGDHIVSASRDKTIKMWEVQTGYCVKTFTGHREWVRMVRPNQDGTLIASCSNDQTVRVWVVATKECKAELREHEHVVECISWAPESSYSSISEATGSETKKSGKPGPFLLSGSRDKTIKMWDVSTGMCLMTLVGHDNWVRGVLFHSGGKFILSCADDKTLRVWDYKNKRCMKTLNAHEHFVTSLDFHKTAPYVVTGSVDQTVKVWECR']
torch.Size([1, 411])
torch.Size([1, 411])
torch.Size([1, 411, 35])
torch.Size([61, 35])
torch.Size([61])
At Epoch: 2001.0
Loss 0.2896
time elapsed 71279.6144
['F4JUI9']
['MEKTQMPVAREEKILVLVRLRPLNQKEIAANEAADWECINDTTILYRNTLREGSNFPSAYSFDKVYRGECPTRQVYEDGTKEIALSVVKGINCSIFAYGQTSSGKTYTMTGITEFAVADIFDYIFQHEERAFSVKFSAIEIYNEAIRDLLSSDGTSLRLRDDPEKGTVVEKATEETLRDWNHLKELLSICEAQRKIGETSLNERSSRSHQMIRLTVESSAREFLGKENSTTLMASVNFIDLAGSERASQAMSAGTRLKEGCHINRSLLTLGTVIRKLSKGRQGHINFRDSKLTRILQPCLGGNARTAIICTLSPARSHVELTKNTLLFACCAKEVTTKARINVVMSDKALLKQLQRELARLETELRNPASSPASNCDCAMTVRKKDLQIQKMEKEIAELRKQR

['Q6AXZ5']
['MGDLELLLPGEAEVLVQGLHSFQLREMGSEGWSKQHENLEKLNMQAILDATVSQAEPIQELLVTHGKIPTLVEELIAVEMWKQKVFPVLCRLEDFKPQNTFPIYMVVHHEASIINLLETVFFHKEVCESADDTVLDLVDYCHRKLILLVARKGGGDLSEEERFQDSTPMQELQKQAEMMEFEISLKALSVLRYITDCMDSLSLSTLNRMLTTHNLPCLLVELLEHSPWSRREGGKLQHFESGRWQTVAPSEHQKLNKLDGQVWIALYNLLLSPEARTRYCLTNFAKGQLLKLQAFLTDTLLDQLPNLADLKSFLAHLALVETQPPKKDLVLEQIPEIWDRLERENKGKWQAIAKHQLQHVFSLSEKDLRQQAQRWAETYRLDVLEAVAPERPRCAYCSAEASKRCSRCQKVWYCCRECQVKHWEKHGKTCVLAAQGDRAK']
torch.Size([1, 441])
torch.Size([1, 441])
torch.Size([1, 441, 35])
torch.Size([66, 35])
torch.Size([66])
At Epoch: 601.0
Loss 0.3423
time elapsed 71652.4576
['P07567']
['MGQELSQHERYVEQLKQALKTRGVKVKYADLLKFFDFVKDTCPWFPQEGTIDIKRWRRVGDCFQDYYNTFGPEKVPVTAFSYWNLIKELIDKKEVNPQVMAAVAQTEEILKSNSQTDLTKTSQNPDLDLISLDSDDEGAKSSSLQDKGLSSTKKPKRFPVLLTAQTSKDPEDPNPSEVDWDGLEDEAAKYHNPDWPPFLTRPPPYNKATPSAPTVMAVVNPKEELKEKIAQLEEQIKLEELHQALISKLQKLKTGNETVTHPDTAGGLSRTPHWPGQHIPKGKCCASREKEEQIPKDIFPVTETVDGQGQAWRHHNGFDFAVIKELKTAASQYGATAPYTLAIVESVADNWLTPTDWNTLVRAVLSGGDHLLWK

['Q9XU39']
['MYAVLLVLALLATAQSGPAQTMPSSTTMTPVAGNSGNSGMQGANTMPMNATGMSNSNSMGPARTSSSTVMPGN']
torch.Size([1, 74])
torch.Size([1, 74])
torch.Size([1, 74, 35])
torch.Size([10, 35])
torch.Size([10])
At Epoch: 1601.0
Loss 0.1119
time elapsed 72062.7814


In [11]:
torch.save(model.state_dict(), "../../data/esm_t12_85M_UR50S_dyn_kin_201204.pt")

In [12]:
print("done")

done
